In [2]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
#Change the date to the previous day
query = """
WITH Schedule as
(
SELECT
SPLIT(Session5Date, ' ')[0] Date, Year, RoundNumber, EventName, EventFormat
FROM `perceptive-ivy-290216.f1_api.schedules_update`
WHERE Session5='Race'
GROUP BY 1,2,3,4,5
ORDER BY 2,1
)
,

Results as (
(
SELECT
DriverNumber, Abbreviation, TeamName,FullName, CAST(Position as INT64) Position,CAST(GridPosition as INT64) GridPosition, Time, Status, CAST(Points as INT64) Points, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat
FROM `perceptive-ivy-290216.f1_api.results_race` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE A.Year=2016
ORDER BY FullName, RoundNumber
)
UNION ALL
(
SELECT
DriverNumber, Abbreviation, TeamName,FullName, CAST(Position as INT64) Position,CAST(GridPosition as INT64) GridPosition, Time, Status, CAST(Points as INT64) Points, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE A.Year=2016
ORDER BY FullName, RoundNumber
)
)

SELECT
Date, DriverNumber, Abbreviation, TeamName,FullName, Position, GridPosition, Time, Status, Points,
SUM(Points) OVER (PARTITION BY Year, FullName ORDER BY ROUNDNUMBER) PTS,
Year, GP, RoundNumber, EventFormat
FROM
Results
ORDER BY FullName, RoundNumber

"""
project_id = "perceptive-ivy-290216"
df_bq = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [5]:
years=2024

In [6]:
df_bq_driver=df_bq[df_bq["Year"]==years]
df_bq_driver=df_bq_driver.sort_values(by=['Date'])
df_bq_driver.head()

,Date,DriverNumber,Abbreviation,TeamName,FullName,Position,GridPosition,Time,Status,Points,PTS,Year,GP,RoundNumber,EventFormat
855,2024-03-02,23,ALB,Williams,Alexander Albon,15,13,NaT,+1 Lap,0,0,2024,Bahrain Grand Prix,1,conventional
5869,2024-03-02,31,OCO,Alpine,Esteban Ocon,17,19,NaT,+1 Lap,0,0,2024,Bahrain Grand Prix,1,conventional
6408,2024-03-02,14,ALO,Aston Martin,Fernando Alonso,9,6,0 days 00:01:14.887000,Finished,2,2,2024,Bahrain Grand Prix,1,conventional
7079,2024-03-02,63,RUS,Mercedes,George Russell,5,3,0 days 00:00:46.788000,Finished,10,10,2024,Bahrain Grand Prix,1,conventional
8137,2024-03-02,24,ZHO,Kick Sauber,Guanyu Zhou,11,17,NaT,+1 Lap,0,0,2024,Bahrain Grand Prix,1,conventional


In [8]:
fig=px.line(
        df_bq_driver, 
        x="GP", 
        y="PTS", 
        color='Abbreviation', 
        # markers=True,
        hover_data=['DriverNumber','TeamName', 'Position', 'GridPosition', 'Status', 'Year', 'GP', 'RoundNumber'], 
        template="xgridoff",
        title="<b>Cumulative Driver Points for the {} F1 Season</b>".format(years),
        height=800, 
        width=1200,
        color_discrete_map={
                "ALB": "#64C4FF",
                "ALO": "#229971",
                "BOT": "#52e252",
                "GAS": "#0093cc",
                "HAM": "#27F4D2",
                "HUL": "#B6BABD",
                "LEC": "#E80020",
                "MAG": "#B6BABD",
                "NOR": "#FF8000",
                "OCO": "#0093cc",
                "PER": "#3671C6",
                "PIA": "#FF8000",
                "RIC": "#6692FF",
                "RUS": "#27F4D2",
                "SAI": "#E80020",
                "SAR": "#37BEDD",
                "STR": "#229971",
                "TSU": "#6692FF",
                "VER": "#3671C6",
                "ZHO": "#52e252"
          },
        )
fig.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=10)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    # margin=dict(l=10, r=5, t=55, b=20),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),

)

In [9]:
fig.write_html("Trended Driver Points for the {} F1 Season.html".format(years))

In [10]:
#Team PTS

In [11]:
#Change the date to the previous day
query = """
WITH Schedule as
(
SELECT
SPLIT(Session5Date, ' ')[0] Date, Year, RoundNumber, EventName, EventFormat
FROM `perceptive-ivy-290216.f1_api.schedules_update`
WHERE Session5='Race'
GROUP BY 1,2,3,4,5
ORDER BY 2,1
)
,

Results as (
(
SELECT
TeamName, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat, SUM(CAST(Points as INT64)) Points
FROM `perceptive-ivy-290216.f1_api.results_race` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
GROUP BY 1,2,3,4,5,6
)
UNION ALL
(
SELECT
TeamName, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat, SUM(CAST(Points as INT64)) Points
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
GROUP BY 1,2,3,4,5,6
)
)

SELECT
TeamName, Year, GP, Date, 
RoundNumber, EventFormat, Points, SUM(Points) OVER (PARTITION BY Year, TeamName ORDER BY ROUNDNUMBER) PTS,
FROM
Results

"""
project_id = "perceptive-ivy-290216"
df_bq_teams = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:2379: UserWarning:

A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.



In [12]:
df_bq_team=df_bq_teams[df_bq_teams["Year"]==years]
df_team=df_bq_team.groupby(by=['Date', 'TeamName','Points', 'Year','GP','RoundNumber','EventFormat'])["PTS"].sum()
df_team=pd.DataFrame(df_team)
df_team.reset_index(inplace=True)
df_team=df_team.sort_values(by=['Date'])
df_team.head()

,Date,TeamName,Points,Year,GP,RoundNumber,EventFormat,PTS
0,2024-03-02,Alpine,0,2024,Bahrain Grand Prix,1,conventional,0
1,2024-03-02,Aston Martin,3,2024,Bahrain Grand Prix,1,conventional,3
2,2024-03-02,Ferrari,27,2024,Bahrain Grand Prix,1,conventional,27
3,2024-03-02,Haas F1 Team,0,2024,Bahrain Grand Prix,1,conventional,0
4,2024-03-02,Kick Sauber,0,2024,Bahrain Grand Prix,1,conventional,0


In [13]:
df_team[df_team['TeamName']=='Ferrari']

,Date,TeamName,Points,Year,GP,RoundNumber,EventFormat,PTS
2,2024-03-02,Ferrari,27,2024,Bahrain Grand Prix,1,conventional,27
12,2024-03-09,Ferrari,22,2024,Saudi Arabian Grand Prix,2,conventional,49
22,2024-03-24,Ferrari,44,2024,Australian Grand Prix,3,conventional,93
32,2024-04-07,Ferrari,27,2024,Japanese Grand Prix,4,conventional,120
44,2024-04-21,Ferrari,22,2024,Chinese Grand Prix,5,sprint_shootout,151
43,2024-04-21,Ferrari,9,2024,Chinese Grand Prix,5,sprint_shootout,151
61,2024-05-05,Ferrari,25,2024,Miami Grand Prix,6,sprint_shootout,187
60,2024-05-05,Ferrari,11,2024,Miami Grand Prix,6,sprint_shootout,187
75,2024-05-19,Ferrari,25,2024,Emilia Romagna Grand Prix,7,conventional,212
85,2024-05-26,Ferrari,40,2024,Monaco Grand Prix,8,conventional,252


In [15]:
fig_team=px.line(
        df_team, 
        x="GP", 
        y="PTS", 
        color='TeamName', 
        # markers=True,
        hover_data=['TeamName','Year', 'GP', 'RoundNumber','EventFormat'], 
        template="xgridoff",
        title="<b>Cumulative Team Points for the {} F1 Season</b>".format(years),
        height=800, 
        width=1200,
        color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 }
        )
fig_team.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=10)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow",
    # margin=dict(l=10, r=5, t=55, b=20),
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow"
    ),

)

In [16]:
fig_team.write_html("Trended Team Points for the {} F1 Season.html".format(years))